In [10]:
cd /Users/tedalcorn/desktop/2024\ 07\ The\ Trace\ -\ Baltimore\ Bar\ Bill/2024\ 07\ 05\ Mapping

/Users/tedalcorn/Desktop/2024 07 The Trace - Baltimore Bar Bill/2024 07 05 Mapping


/Users/tedalcorn/.pyenv/versions/3.11.8/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
pwd

'/Users/tedalcorn/Desktop/2024 07 The Trace - Baltimore Bar Bill/2024 07 05 Mapping'

In [7]:
import csv
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import re

In [12]:
with open('240520_Baltimore_master.csv', newline='') as fp:
    reader = csv.DictReader(fp)
    data = list(reader)

# Display the first three items
print(data[:3])

[{'\ufeffBlock': '16-Dec', 'Street': '20TH STREET WEST', 'Address': '12-16 20TH STREET WEST', 'Zip': '21218', 'Trade Name': 'KONG POCHA', 'Corp Name': 'SOY BEAN, INC.', 'License': 'LB'}, {'\ufeffBlock': '225-27', 'Street': '23RD STREET WEST', 'Address': '225-27 23RD STREET WEST', 'Zip': '21211', 'Trade Name': 'CLAVEL', 'Corp Name': 'HARLAN, INC', 'License': 'LB'}, {'\ufeffBlock': '400', 'Street': '23RD STREET WEST', 'Address': '400 23RD STREET WEST', 'Zip': '21211', 'Trade Name': 'W.C. HARLAN', 'Corp Name': '400 W. 23RD STREET, INC.', 'License': 'LD'}]


In [14]:
# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Create full address
df['FullAddress'] = df['Address'] + ', Baltimore, MD, ' + df['Zip'] + ', USA'

# Display the DataFrame with the FullAddress column
print(df[['FullAddress']])

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1170    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1171      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1172     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  
0                      KONG POCHA    

In [15]:
len(df)

1173

In [6]:
# Function to move directional indicators
def move_direction_to_start(address):
    # Pattern to match a directional indicator at the end of the street name
    pattern = re.compile(r'\b(North|South|East|West)\b', flags=re.IGNORECASE)
    
    # Check if there's a match for a directional indicator at the end
    match = pattern.search(address)
    if match:
        direction = match.group(0)
        # Remove directional indicator from the end
        address = pattern.sub('', address).strip()
        # Split address into parts at the first space
        parts = address.split(' ', 1)
        if len(parts) == 2:
            # Reconstruct address with direction after the first part (street number)
            address = f"{parts[0]} {direction} {parts[1]}"
        else:
            address = f"{address} {direction}"
    
    return address

# Function to remove second number in hyphenated address
def remove_second_number(address):
    return re.sub(r'(\d+)-\d+', r'\1', address, flags=re.IGNORECASE)

# Function to remove suite or unit information
def remove_suite_or_unit(address):
    return re.sub(r'\s+(Suite|Unit)\s+\d+', '', address, flags=re.IGNORECASE)

# Function to clean addresses
def clean_address(address):
    address = move_direction_to_start(address)
    address = remove_second_number(address)
    address = remove_suite_or_unit(address)
    return address

In [7]:
# Apply the cleaning function to the FullAddress column
df['CleanedAddress'] = df['FullAddress'].apply(clean_address)

# Print the cleaned addresses
print(df)

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1170    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1171      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1172     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  \
0                      KONG POCHA   

In [8]:
# Example with progress tracking
total_addresses = len(df)
for i, address in enumerate(df['FullAddress']):
    df.at[i, 'CleanedAddress'] = clean_address(address)
    if i % 100 == 0:  # Print progress every 100 addresses
        print(f"Processed {i+1}/{total_addresses} addresses")

# Print the cleaned addresses
print(df)

Processed 1/1173 addresses
Processed 101/1173 addresses
Processed 201/1173 addresses
Processed 301/1173 addresses
Processed 401/1173 addresses
Processed 501/1173 addresses
Processed 601/1173 addresses
Processed 701/1173 addresses
Processed 801/1173 addresses
Processed 901/1173 addresses
Processed 1001/1173 addresses
Processed 1101/1173 addresses
       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET E

In [9]:
# Apply the cleaning function to the FullAddress column
df['CleanedAddress'] = df['FullAddress'].apply(clean_address)

# Print the cleaned addresses to verify
print(df[['FullAddress', 'CleanedAddress'])

                                            FullAddress  \
0     12-16 20TH STREET WEST, Baltimore, MD, 21218, USA   
1     225-27 23RD STREET WEST, Baltimore, MD, 21211,...   
2       400 23RD STREET WEST, Baltimore, MD, 21211, USA   
3       101 25TH STREET EAST, Baltimore, MD, 21218, USA   
4      1001 25TH STREET EAST, Baltimore, MD, 21218, USA   
...                                                 ...   
1168  6726 YOUNGSTOWN AVENUE, Baltimore, MD, 21222, USA   
1169  201-207 REDWOOD STREET EAST, Baltimore, MD, 21...   
1170  2-4 PRESTON STREET EAST, Baltimore, MD, 21202,...   
1171    720 ALICEANNA STREET, Baltimore, MD, 21202, USA   
1172  1200 NORTH AVENUE WEST, Baltimore, MD, 21217, USA   

                                         CleanedAddress  
0       12 WEST 20TH STREET , Baltimore, MD, 21218, USA  
1      225 WEST 23RD STREET , Baltimore, MD, 21211, USA  
2      400 WEST 23RD STREET , Baltimore, MD, 21211, USA  
3      101 EAST 25TH STREET , Baltimore, MD, 21218, USA  
4

In [1]:
# Initialize Nominatim geocoder with a unique user-agent
geolocator = Nominatim(user_agent="baltimore_alcohol")

# List to store geocoded locations
locations = []

# Function to handle geocoding with retries
def geocode_address(address, retries=3):
    try:
        location = geolocator.geocode(address, timeout=10)
        return location
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        if retries > 0:
            time.sleep(1)  # Wait before retrying
            return geocode_address(address, retries - 1)
        else:
            print(f"Error geocoding {address}: {e}")
            return None

# Process addresses in batches
batch_size = 100
for i in range(0, len(df), batch_size):
    batch = df['CleanedAddress'][i:i + batch_size]
    for address in batch:
        location = geocode_address(address)
        if location:
            locations.append((address, location.latitude, location.longitude))
    print(f"Processed batch {i // batch_size + 1}")
    time.sleep(5)  # Delay between batches to respect rate limits

# Print geocoded locations
for address, lat, lon in locations[:10]:
    print(f"Address: {address}\nLatitude: {lat}\nLongitude: {lon}\n")

NameError: name 'Nominatim' is not defined

In [ ]:
# Convert locations list to DataFrame
df_locations = pd.DataFrame(locations, columns=['CleanedAddress', 'Latitude', 'Longitude'])

# Step 3: Merge DataFrames based on FullAddress
df_merged = pd.merge(df, df_locations, on='CleanedAddress', how='left')

# Step 4: Save the merged DataFrame back to a new CSV file
df_merged.to_csv('merged_with_locations.csv', index=False)

# Print the merged DataFrame for verification
print(df_merged)

In [ ]:
# Drop duplicates based on Address and Trade Name
df_unique = df_merged.drop_duplicates(subset=['Address', 'Trade Name'])
len(df_unique)

In [18]:
# Manually cleaned data

# Load the CSV into a DataFrame
df = pd.read_csv('merged_with_locations.csv')

# Filter rows where 'ManualCleaned' is not null
df_to_geocode = df[df['ManualCleaned'].notnull()]

# Initialize Nominatim geocoder with a unique user-agent
geolocator = Nominatim(user_agent="baltimore_alcohol")

# Function to handle geocoding with retries
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        return location
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        print(f"Error geocoding {address}: {e}")
        return None

# Geocode addresses and update 'Latitude' and 'Longitude' columns directly in DataFrame
for idx, row in df_to_geocode.iterrows():
    if pd.notnull(row['ManualCleaned']):
        address = row['ManualCleaned']
        location = geocode_address(address)
        if location:
            df.loc[idx, 'Latitude'] = location.latitude
            df.loc[idx, 'Longitude'] = location.longitude

# Save the updated DataFrame back to CSV
df.to_csv('merged_with_locations.csv', index=False)

print("Geocoding and update completed. Check 'merged_with_locations.csv' for updated data.")

Geocoding and update completed. Check 'merged_with_locations.csv' for updated data.


In [22]:
# Drop duplicates based on Address and Trade Name
df_unique = df.drop_duplicates(subset=['Address', 'Trade Name'])
len(df_unique)

1172

In [23]:
len(df)

1233

In [32]:
# # Remove unwanted columns: 'Block', 'Street', 'Address', 'Zip'
# columns_to_drop = ['Block', 'Street', 'Address', 'Zip']
# df_unique.drop(columns_to_drop, axis=1, inplace=True)

# Save the DataFrame to a new CSV file
df_unique.to_csv('geocoded_unique.csv', index=False)

In [29]:
len(df_unique)

1172

In [30]:
df_unique.columns

Index(['Trade Name', 'Corp Name', 'License', 'FullAddress', 'CleanedAddress',
       'ManualCleaned', 'Latitude', 'Longitude'],
      dtype='object')